# 1861 Hagelloch Measles

Data references:

* Pfeilsticker, A. 1863. Beiträge zur Pathologie der Masern mit besonderer Berücksichtigung der statistischen Verhältnisse, M.D. Thesis, Eberhard-Karls-Universität Tübingen. Available as http://www.archive.org/details/beitrgezurpatho00pfeigoog.
* Oesterle, H. 1992. Statistische Reanalyse einer Masernepidemie 1861 in Hagelloch, M.D. Thesis, Eberhard-Karls-Universitäat Tübingen.
* Höhle M. 2007. surveillance: An R package for the monitoring of infectious diseases. Computational Statistics, 22:571-582.

In [ ]:
using CSV, DelimitedFiles, Distances, Random, Pathogen, Plots, Plots.PlotMeasures, DataFrames
Random.seed!(4321)

In [2]:
# POPULATION INFOFORMATION
# Use CSV.jl for DataFrames I/O
risks = CSV.read(joinpath(@__DIR__, "data/measles_hagelloch_1861_risk_factors.csv"))

,age,gender,family_ID,class,x,y
,Int64,String,Int64,Int64,Float64,Float64
1,7,f,41,1,142.5,100.0
2,6,f,41,1,142.5,100.0
3,4,f,41,0,142.5,100.0
4,13,m,61,2,165.0,102.5
5,8,f,42,1,145.0,120.0
6,12,m,42,2,145.0,120.0
7,6,m,26,0,272.5,147.5
8,10,m,44,1,97.5,155.0
9,13,m,44,2,97.5,155.0


In [3]:
# Will precalculate distances
distances = [euclidean([risks[i, :x]; risks[i, :y]], [risks[j, :x]; risks[j, :y]]) for i = 1:size(risks, 1), j = 1:size(risks, 1)]

pop = Population(risks, distances)

Population object (n=188)

In [3]:
# Define risk functions/TN-ILM structure
function _constant(params::Vector{Float64}, pop::Population, i::Int64)
  return params[1]
end

function _one(params::Vector{Float64}, pop::Population, i::Int64)
  return 1.0
end

function _powerlaw_plus(params::Vector{Float64}, pop::Population, i::Int64, k::Int64)    
  return distances[k,i] > 0.0 * params[1] * pop.distances[k, i]^(-params[2]) +
         params[3]*(pop.risks[i, :family_ID] == pop.risks[k, :family_ID]) +
         params[4]*(pop.risks[i, :class] == pop.risks[k, :class])
end

rf = RiskFunctions{SEIR}(_constant, # sparks function
                         _one, # susceptibility function
                         _powerlaw_plus, # infectivity function
                         _one, # transmissability function
                         _constant, # latency function
                         _constant) # removal function

SEIR model risk functions

In [6]:
obsdata = CSV.read(joinpath(@__DIR__, "data/measles_hagelloch_1861_observations.csv"))

# Set the removal observation as minimum of (day that rash appears + 4.0) and death, in fatal cases.
removed = [obsdata[i, :death] === NaN ? obsdata[i, :rash] + 4.0 : min(obsdata[i, :rash] + 4.0, obsdata[i, :death]) for i = 1:188]

# Set prodrome within first 7 days of epidemic as initial conditions
infected = obsdata[:, :prodrome]
# starting_states = [i <= 10.0 ? State_I : State_S for i in infected]
# infected[infected .<= 10.0] .= -Inf
obs = EventObservations{SEIR}(infected, removed)

SEIR model observations (n=188)

In [6]:
# Specify some priors for the risk parameters of our various risk functions
rpriors = RiskPriors{SEIR}([Uniform(0.0, 1.0)],
                           UnivariateDistribution[],
                           [Uniform(1.0, 7.0); Uniform(1.0, 7.0); Uniform(0.0, 2.0); Uniform(0.0, 2.0)],
                           UnivariateDistribution[],
                           [Uniform(0.0, 1.0)],
                           [Uniform(0.0, 1.0)])

# Using CDC measles information set some extents for event data augmentation
# Exposure up to 2 weeks before infectiousness
# Infectious up to 3 days before prodrome
# Removal time within 2-4 days after rash
ee = EventExtents{SEIR}(14.0, 3.0, 2.0)

SEIR model event extents

In [8]:
# Initialize MCMC
mcmc = MCMC(obs, ee, pop, rf, rpriors)
start!(mcmc, attempts=50000) # 1 chain, with 50k initialization attempts

# Run MCMC
iterate!(mcmc, 100000, 1.0, condition_on_network=true, event_batches=5)

Initialization progress100%|████████████████████████████| Time: 0:09:19m
MCMC progress100%|██████████████████████████████████████| Time: 0:58:31m


SEIR model MCMC with 1 chains

In [22]:
# using JLD2
# @save "mcmc.jld2" mcmc
# @load "mcmc.jld2" mcmc

In [12]:
# MCMC and posterior plots
p1 = plot(1:20:100001, mcmc.markov_chains[1].risk_parameters, yscale=:log10, title="TN-ILM parameters")
png(p1, joinpath(@__DIR__, "trace.png"))

![TN-ILM traceplot](trace.png)

In [8]:
gr(dpi=200)

p1 = plot(mcmc.markov_chains[1].events[50000], State_S,
          linealpha=0.01, title="S", xguidefontsize=8, yguidefontsize=8,
          xtickfontsize=7, ytickfontsize=7, titlefontsize=11)
for i=50050:50:100000
  plot!(p1, mcmc.markov_chains[1].events[i], State_S, linealpha=0.02)
end

p2 = plot(mcmc.markov_chains[1].events[50000], State_E,
          linealpha=0.01, title="E", xguidefontsize=8, yguidefontsize=8,
          xtickfontsize=7, ytickfontsize=7, titlefontsize=11)
for i=50050:50:100000
  plot!(p2, mcmc.markov_chains[1].events[i], State_E, linealpha=0.02)
end

p3 = plot(mcmc.markov_chains[1].events[50000], State_I,
          linealpha=0.01, title="I", xguidefontsize=8, yguidefontsize=8, xtickfontsize=7, ytickfontsize=7, titlefontsize=11)
for i=50050:50:100000
  plot!(p3, mcmc.markov_chains[1].events[i], State_I, linealpha=0.02)
end
plot!(p3, obs, State_I, linecolor=:black, linewidth=1.5) # Show infection observations (day of prodrome)

p4 = plot(mcmc.markov_chains[1].events[50000], State_R,
          linealpha=0.01, title="R", xguidefontsize=8, yguidefontsize=8, xtickfontsize=7, ytickfontsize=7, titlefontsize=11)
for i=50050:50:100000
  plot!(p4, mcmc.markov_chains[1].events[i], State_R, linealpha=0.02)
end
plot!(p4, obs, State_R, linecolor=:black, linewidth=1.5) # Show removal observations (day of appearance of rash + 4)

l = @layout [a b; c d]
combinedplots2 = plot(p1, p2, p3, p4, layout=l)
png(combinedplots2, joinpath(@__DIR__, "posterior_epi_curves.png"))

![Epidemic Curve Posterior](posterior_epi_curves.png)

In [16]:
tnp = TransmissionNetworkPosterior(mcmc.markov_chains[1].transmission_network[50000:50:100000])
p1 = plot(tnp, mcmc.population, title="Transmission Network\nPosterior Distribution", titlefontsize=11, framestyle=:box)

png(p1, joinpath(@__DIR__, "posterior_tn.png"))

In [17]:
# As there are several individuals at single locations, jitter locations to better illustrate the 
# transmission network distribution
xyjitter = select(risks, :x, :y)
xyjitter[:, :x] = xyjitter[:, :x] + rand(Normal(0, 5), 188)
xyjitter[:, :y] = xyjitter[:, :y] + rand(Normal(0, 5), 188)
plotpop = Population(xyjitter)

Population object (n=188)

In [18]:
p1 = plot(tnp, plotpop, title="Transmission Network\nPosterior Distribution", titlefontsize=11, framestyle=:box, markeralpha=0.5)

png(p1, joinpath(@__DIR__, "posterior_tn2.png"))

In [32]:
tntrue = TransmissionNetwork(BitArray(readdlm("data/measles_hagelloch_1861_oesterle_tn_external.csv")[:]), BitArray(readdlm("data/measles_hagelloch_1861_oesterle_tn_internal.csv")))
p2 = plot(tntrue, plotpop, title="Oesterle (1992) Transmission Network", titlefontsize=11, framestyle=:box, markeralpha=0.5)

png(p2, joinpath(@__DIR__, "Oesterle_tn.png"))

In [33]:
l = @layout [a b]
plot(p1, p2, layout=l, dpi=200)
png(joinpath(@__DIR__, "tn_side_by_side.png"))

![Transmission Posterior](tn_side_by_side.png)

In [29]:
# Get a summary of TN-ILM parameters
summary(mcmc, burnin=50000, thin=50)

,parameter,mean,var,credible99,credible95
,String,Float64,Float64,Tuple…,Tuple…
1,ϵ₁,0.000804488,1.40838e-7,"(0.000131699, 0.0019919)","(0.000220164, 0.00166633)"
2,κ₁,3.93305,2.92993,"(1.05857, 6.95224)","(1.15176, 6.79946)"
3,κ₂,4.17346,3.12885,"(1.06592, 6.9752)","(1.22787, 6.90885)"
4,κ₃,0.9891,0.350459,"(0.0229848, 1.99359)","(0.0586367, 1.96378)"
5,κ₄,0.963179,0.311274,"(0.0172885, 1.97019)","(0.0780161, 1.92117)"
6,Ωl₁,0.103978,5.59082e-5,"(0.0861229, 0.122839)","(0.0896032, 0.11895)"
7,Ωr₁,0.122385,7.00205e-5,"(0.102961, 0.142458)","(0.10609, 0.140014)"
